# C

## 1 

In [54]:
import pandas as pd
import plotly.express as px 
import numpy as np
import scipy.stats as ss
from scipy.stats import chi2_contingency


In [55]:
matches = pd.read_csv("data/matches_combined_data.csv")
tour = pd.read_csv('data/fjelstul/tournaments.csv')


merge away and home teams

In [56]:

matches['teams']= matches.apply(lambda row : ','.join(str(item) for item in sorted([row.home_team_name,row.away_team_name])),axis=1)

In [57]:
ten_most_played = matches.groupby('teams').size().nlargest(10)

## 2

In [58]:
px.bar(ten_most_played)

we notice that Barazil and Argentina have played with many teams because they have won many tournemants
 

# D

## 1

In [59]:
players_teams_agg = pd.read_csv('data/player_teams_agg.csv')

In [60]:
players_teams_agg[['player_id','given_name','family_name','team_name','team_count']][players_teams_agg.team_count > 1]

,player_id,given_name,family_name,team_name,team_count
455,P-00537,Franz,Wagner,"Austria,Germany",2
1080,P-01259,Ferenc,Puskás,"Hungary,Spain",2
1298,P-01512,José,Altafini,"Brazil,Italy",2
1490,P-01739,Davor,Šuker,"Yugoslavia,Croatia",2
1502,P-01757,Rudolf,Raftl,"Austria,Germany",2
2016,P-02369,Dejan,Stanković,"Yugoslavia,Serbia and Montenegro,Serbia",3
2129,P-02502,Nikola,Žigić,"Serbia and Montenegro,Serbia",2
2163,P-02543,Luis,Monti,"Argentina,Italy",2
2191,P-02576,Robert,Prosinečki,"Yugoslavia,Croatia",2
2416,P-02845,Attilio,Demaría,"Argentina,Italy",2


Some teams have been finished because their countries don't exsist anymore like  Soviet Union countries, East and West Germany, also almost the same in Serbia and Montenegro,Serbia.

Or if the player has many citizenship like José Altafini

## 2

In [61]:
wh = tour[['winner','host_country']]

In [62]:
wh

,winner,host_country
0,Uruguay,Uruguay
1,Italy,Italy
2,Italy,France
3,Uruguay,Brazil
4,West Germany,Switzerland
5,Brazil,Sweden
6,Brazil,Chile
7,England,England
8,Brazil,Mexico
9,West Germany,West Germany


In [63]:
winner_is_host = wh[wh.winner == wh.host_country]
winner_is_host

,winner,host_country
0,Uruguay,Uruguay
1,Italy,Italy
7,England,England
9,West Germany,West Germany
10,Argentina,Argentina
15,France,France


In [64]:
winner_count = tour.groupby('winner').size()
winner_count

winner
Argentina       3
Brazil          5
England         1
France          2
Germany         1
Italy           4
Spain           1
Uruguay         2
West Germany    3
dtype: int64

In [65]:
winner_is_host_with_count = wh.merge(winner_count.to_frame(name='winner_count'),on='winner',how="left")
winner_is_host_with_count.sort_values(by='winner')

,winner,host_country,winner_count
10,Argentina,Argentina,3
12,Argentina,Mexico,3
21,Argentina,Qatar,3
5,Brazil,Sweden,5
6,Brazil,Chile,5
8,Brazil,Mexico,5
14,Brazil,United States,5
16,Brazil,"Korea, Japan",5
7,England,England,1
20,France,Russia,2


In [66]:
def cramers_v(cross_tabs):
    """
    Prints the degrees of freedom, effect size thresholds, and Cramer's V value.
    
    Args:
        cross_tabs: A crosstab dataframe.
    """
    
    # getting the chi sq. stat
    chi2 = ss.chi2_contingency(cross_tabs)[0]    # calculating the total number of observations
    n = cross_tabs.sum().sum()    # getting the degrees of freedom
    dof = min(cross_tabs.shape)-1    # calculating cramer's v
    v = np.sqrt(chi2/(n*dof))    # printing results
    print(f'V = {v}')
    print(f'Cramer\'s V Degrees of Freedom = {dof}')
    
    

In [67]:
def print_chi2_p_cramers(df,c1,c2):
    c = df[[c1,c2]]
    contingency_table = pd.crosstab(c[c1], c[c2])
    chi2, p, *_ = chi2_contingency(contingency_table)
    print('Chi-square statistic:', chi2)
    print('P-value:', p)
    cramers_v(contingency_table)


In [68]:
print_chi2_p_cramers(tour,'winner','host_country')

Chi-square statistic: 138.96666666666667
P-value: 0.41334333987682675
V = 0.8885850175044216
Cramer's V Degrees of Freedom = 8


There is no clear relation, except that England is suspicious

V is almost 1 which means there is a strog relation according to cramers'V test 

also it is clear that some country didn't won except for when they were the host 

## 3

In [69]:
print_chi2_p_cramers(matches,'relative_attendance_category','match_for_host')

Chi-square statistic: 3.797195837618871
P-value: 0.14977847365244198
V = 0.06276145179610527
Cramer's V Degrees of Freedom = 1


# 4

In [70]:
print_chi2_p_cramers(matches,'attendance_category','host_country_code')

Chi-square statistic: 646.0450125820513
P-value: 4.224118972361396e-95
V = 0.4093199161040318
Cramer's V Degrees of Freedom = 4
